In [27]:
import pandas as pd
import numpy as np

In [28]:
# Cargar el dataset
df = pd.read_csv('./data_versions/1_initial_csv.csv')

type_to_incident_mapping = {
    'Unprovoked': 'Unprovoked Incidents',
    ' Provoked': 'Provoked Incidents',
    'Provoked': 'Provoked Incidents',
    'Questionable': 'Questionable Incidents',
    'Watercraft': 'Attacks on Boats',
    'Sea Disaster': 'Air/Sea Disasters',
    '?': 'Questionable Incidents',
    'Unconfirmed': 'Questionable Incidents',
    'Unverified': 'Questionable Incidents',
    'Invalid': 'Questionable Incidents',
    'Under investigation': 'Questionable Incidents',
    'Boat': 'Attacks on Boats',
    np.nan: np.nan
}

def perfilado_datos(df):

    n_registers = df.shape[0]
    print("Cantidad de registros:", n_registers)
    print("1. Completitud:")
    missing_type = (df['Incident Type'].isnull() & df['Type'].isnull()).sum()
    coherent_flags = df.apply(lambda row: row['Incident Type'] == type_to_incident_mapping.get(row['Type']), axis=1)

    # Calculate the proportion of incoherent incidents
    incoherent_proportion = 1 - coherent_flags.mean()
    missing_country = df['Country'].isnull().sum()
    print(f" - Registros sin información de tipo de incidente: {missing_type/n_registers:.2%}")
    print(f" - Registros con tipos de incidente incoherentes: {incoherent_proportion:.2%}")
    print(f" - Registros sin país asignado: {missing_country/ n_registers:.2%}")
    
    print("\n2. Unicidad:")
    columnas_clave = ['Date', 'Location', 'Name']  # Ajusta si es necesario
    duplicados = df.duplicated(subset=columnas_clave).sum()
    print(f" - Cantidad de registros duplicados en las columnas clave {columnas_clave}: {duplicados / n_registers:.2%}")
    
    print("\n3. Consistencia en nombres de países:")
    df['Country'] = df['Country'].str.upper().str.strip()  # Estandarizar a mayúsculas y eliminar espacios
    print(" - Nombres de países únicos (primeros 10):", df['Country'].unique()[:10])
    
    
    lista_paises_estandar = ["USA", "AUSTRALIA", "FRANCE", "INDONESIA", "HAWAII"]  # Lista parcial de ejemplo
    paises_invalidos = df[~df['Country'].isin(lista_paises_estandar)]['Country'].unique()
    print("\n4. Países no coincidentes con la lista estándar:")
    print(" - Países fuera de lista estándar:", paises_invalidos)
    
    ocean_keywords = ['OCEAN', 'SEA', 'INTERNATIONAL']
    ubicaciones_oceano = df[df['Location'].str.upper().str.contains('|'.join(ocean_keywords), na=False)]
    print(f"\n5. Registros en ubicaciones de océanos o aguas internacionales: {len(ubicaciones_oceano)}")
    print(" - Ejemplos de ubicaciones en aguas internacionales:", ubicaciones_oceano['Location'].unique()[:10])
    
    print("\n6. Verificación de formato de fecha (YYYY-MM-DD):")
    try:
        df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
        fechas_invalidas = df['Date'].isnull().sum()
        print(f" - Fechas con formato inválido: {fechas_invalidas}")
    except Exception as e:
        print(" - Error al analizar el formato de fechas:", e)
    
    print("\n7. Detección de inconsistencias en nombres de país:")
    inconsistencias_paises = df.groupby('Location')['Country'].nunique().reset_index()
    inconsistencias_paises = inconsistencias_paises[inconsistencias_paises['Country'] > 1]
    print(f" - Cantidad de ubicaciones con nombres de país inconsistentes: {len(inconsistencias_paises)}")
    if not inconsistencias_paises.empty:
        print(" - Ejemplos de ubicaciones con nombres de país inconsistentes:")
        print(inconsistencias_paises.head(10))


In [29]:
perfilado_datos(df)

Cantidad de registros: 6994
1. Completitud:
 - Registros sin información de tipo de incidente: 0.26%
 - Registros con tipos de incidente incoherentes: 1.30%
 - Registros sin país asignado: 0.97%

2. Unicidad:
 - Cantidad de registros duplicados en las columnas clave ['Date', 'Location', 'Name']: 0.53%

3. Consistencia en nombres de países:
 - Nombres de países únicos (primeros 10): ['HAWAII' 'AUSTRALIA' 'USA' 'HONDURAS' 'MOROCCO' 'JAMAICA' 'BELIZE'
 'MALDIVE ISLANDS' 'TURKS AND CAICOS' 'FRENCH POLYNESIA']

4. Países no coincidentes con la lista estándar:
 - Países fuera de lista estándar: ['HONDURAS' 'MOROCCO' 'JAMAICA' 'BELIZE' 'MALDIVE ISLANDS'
 'TURKS AND CAICOS' 'FRENCH POLYNESIA' 'TOBAGO' 'BAHAMAS' 'INDIA'
 'TRINIDAD' 'SOUTH AFRICA' 'MEXICO' 'NEW ZEALAND' 'EGYPT' 'PHILIPPINES'
 'CORAL SEA' 'SPAIN' 'PORTUGAL' 'SAMOA' 'COLOMBIA' 'ECUADOR'
 'NEW CALEDONIA' 'CUBA' 'BRAZIL' 'SEYCHELLES' 'ARGENTINA' 'FIJI'
 'MALDIVES' 'ENGLAND' 'JAPAN' 'THAILAND' 'COLUMBIA' 'COSTA RICA'
 'BRITISH OVERSE

In [15]:
df["Incident Type"].unique()

array(['Air/Sea Disasters', 'Unprovoked Incidents', 'Provoked Incidents',
       'Questionable Incidents', 'Attacks on Boats', nan], dtype=object)